In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.25 already installed.


## Download dataset

Original VLBA dataset gotten from https://casaguides.nrao.edu/index.php?title=VLBA_Basic_Phase-referencing_Calibration_and_Imaging.

Reduced the data using:

```Python
mstransform(vis='VLBA_TL016B.ms/',outputvis='VLBA_TL016B_split_lsrk.ms',spw='0:0~5,1:0~5',field='0,1', timerange='2022/02/21/06:14:00~2022/02/21/07:55:45.00', regridms=True,outframe='lsrk',datacolumn='all')
```




In [2]:
import graphviper

graphviper.utils.data.download(file="VLBA_TL016B_split.ms")

[2024-04-11 10:43:27,403]     INFO  graphviper:  Couldn't find file metadata locally in /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox/file.download.json, checking remote API ... 


VLBA_TL016B_split.ms.zip:   0%|          | 0.00/8.97M [00:00<?, ?iB/s]

[2024-04-11 10:43:29,261]     INFO  graphviper:  Extracting file: VLBA_TL016B_split.ms.zip ... 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "VLBA_TL016B_split.ms"
partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "VLBA_TL016B_split_lsrk.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency,shape,field_coords
0,VLBA_TL016B_split_lsrk_ddi_1_intent_None_field...,1,None,1,J1154+6022,5.068000e+09,5.070500e+09,"(540, 55, 6, 2)","[fk5, 11h54m04.535308s, 60d22m20.81576s]"
1,VLBA_TL016B_split_lsrk_ddi_1_intent_None_field...,1,None,0,4C39.25,5.068000e+09,5.070500e+09,"(200, 55, 6, 2)","[fk5, 9h27m03.0139414s, 39d02m20.851807s]"
2,VLBA_TL016B_split_lsrk_ddi_0_intent_None_field...,0,None,0,4C39.25,5.004000e+09,5.006500e+09,"(200, 55, 6, 2)","[fk5, 9h27m03.0139414s, 39d02m20.851807s]"
3,VLBA_TL016B_split_lsrk_ddi_0_intent_None_field...,0,None,1,J1154+6022,5.004000e+09,5.006500e+09,"(540, 55, 6, 2)","[fk5, 11h54m04.535308s, 60d22m20.81576s]"


In [5]:
ps.keys()

dict_keys(['VLBA_TL016B_split_lsrk_ddi_1_intent_None_field_id_1', 'VLBA_TL016B_split_lsrk_ddi_1_intent_None_field_id_0', 'VLBA_TL016B_split_lsrk_ddi_0_intent_None_field_id_0', 'VLBA_TL016B_split_lsrk_ddi_0_intent_None_field_id_1'])

In [6]:
ps["VLBA_TL016B_split_lsrk_ddi_1_intent_None_field_id_1"]

<xarray.Dataset> Size: 6MB
Dimensions:                     (time: 540, baseline_id: 55, frequency: 6,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 220B dask.array<chunksize=(55,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 220B dask.array<chunksize=(55,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 440B 0 1 2 3 ... 51 52 53 54
  * frequency                   (frequency) float64 48B 5.068e+09 ... 5.07e+09
  * polarization                (polarization) <U2 16B 'RR' 'LL'
  * time                        (time) float64 4kB 1.645e+09 ... 1.645e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 238kB dask.array<chunksize=(20, 55), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 356kB dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 238kB dask.array<chunksize=(20, 55), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 713kB dask.array<chunksize=(20, 55, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 3MB dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 1MB dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
Attributes:
    data_groups:   {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VIS...
    ddi:           1
    intent:        None
    antenna_xds:   <xarray.Dataset> Size: 2kB\nDimensions:        (antenna_id...
    weather_xds:   <xarray.Dataset> Size: 486kB\nDimensions:         (station...
    pointing_xds:  <xarray.Dataset> Size: 0B\nDimensions:  ()\nData variables...

In [7]:
ps["VLBA_TL016B_split_lsrk_ddi_1_intent_None_field_id_1"].attrs["antenna_xds"]

<xarray.Dataset> Size: 2kB
Dimensions:        (antenna_id: 10, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 80B 0 1 2 3 4 5 6 7 8 9
    mount          (antenna_id) <U6 240B dask.array<chunksize=(10,), meta=np.ndarray>
    name           (antenna_id) <U2 80B dask.array<chunksize=(10,), meta=np.ndarray>
    station        (antenna_id) <U2 80B dask.array<chunksize=(10,), meta=np.ndarray>
    type           (antenna_id) <U12 480B dask.array<chunksize=(10,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 12B 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 80B dask.array<chunksize=(10,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 240B dask.array<chunksize=(10, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 240B dask.array<chunksize=(10, 3), meta=np.ndarray>